# <center> Ансамбли </center>

`Дано:` 
- множество объектов обучающей выборки $X^l = (x_i, y_i)_{i=1}^{l}, x_i\in \mathbb{R}$, разделенных на классы $Y=\{-1,1\}$; 
- множество объектов тестовой выборки, классовая принадлежность которых не известна. 
- $a_t: X\rightarrow Y, t=1,...T$ - обучаемые базовые алгоритмы.

**Алгоритм классификации имеет вид:** 
$$a_t(x)=C(b_t(x))$$
$$a_t: X \overset{b_t}{\rightarrow}R\overset{C}{\rightarrow}Y$$
- $b_t$ - алгоритмический оператор (вероятность принадлежности объекта к классу, расстояние объекта до разделяющей поверхности, степень уверенности классификации и т.п.

*Композиция $T$ алгоритмов $a_t(x)=C(b_t(x)), t=1,...T$* называется суперпозиция алгоритмических операторов $b_t:X\rightarrow R$, корректирующей операции $F:R^T \rightarrow R$ и решающего правила $C:R\rightarrow Y$.

$$a(x)=C(F(b_1(x),...,b_T(X))), x \in X$$

**Примеры агрегирующих функций:** 
- простое голосование
$$F(b_1,...,b_T) = \frac{1}{T}\sum_{t=1}^T b_t$$
- взвешенное голосование 
$$F(b_1,...,b_T)= \sum_{t=1}^T \alpha_t b_t, \quad \sum_{t=1}^T\alpha_t=1, \quad \alpha_t  \geq 0$$
- смесь алгоритмов с функциями компетентности 
$$F(b_1,...,b_T)= \sum_{t=1}^T g_t(x) b_t$$

**Базовые алгоритмы не являются независимыми:** 
- решают одну и ту же задачу;
- настраиваются на один целевой вектор ($y_i$);
- выбираются из одной и той же модели.

**Способы повышения разнообразия базовых алгоритмов:** 
- обучение по различным подвыборкам; 
- обучение по различным наборам признаков; 
- обучение из разных моделей; 
- обучение по с использованием рандомизации: 
    - bagging - формирование подвыборок из обучающей выборки "с возвращением" (длина каждой подвыборки равна длине исходной выборки);
    - pasting - формирование случайной обучающей подвыборки;
    - random subspaces - формирование случайного подмножества признаков;
    - random patches - формирование случайного подмножества и объектов, и признаков;
    - cross-validated committees. 

## Бустинг для задачи классификации

Предположим:
- решающее правило фиксировано $C=sign(b)$;
- $b_t: X \rightarrow \{-1,0,1\}$, $b_t(x)=0$ отказ базового алгоритма от классификации объекта $x$.

Алгоритмическая композиция имеет вид:
$$a(x)=C(F(b_1(x),...,b_T(X)))=sign(\sum_{t=1}^T \alpha_t b_t(x)), x \in X$$

Функционал качества композиции -- количество ошибок на выборке:
$$Q_T = \sum_{i=1}^l [y_i*\sum_{t=1}^T \alpha_t b_t(x)<0]$$

Для упращения задачи минимизации фукнционала $Q_T$ используем следующие эвристики:
- При добавлении в композицию слагаемого $\alpha_tb_t(x)$ оптимизируется только базовый алгоритм $b_t$ и коэффициент при нем $\alpha_t$, а все предыдущие слагаемые $\alpha_1b_1(x),...,\alpha_{t-1}b_{t-1}(x)$ полагаются фиксированными.
- Порогавая функция потерь в функционале $Q_t$ аппроксимируется (заменяется) непрерывно дифференцируемой оценкой сверху. Для adaboost используется экспоненциальная аппроксимация: $[y_ib(x_i)<0]\leq e^{-y_ib(x_i)}$

Тогда функционал $Q_t$ можно оценить сверху:

$$Q_T \leq \widetilde{Q_T} = \sum_{i=1}^l exp(-y_i\sum_{t=1}^T \alpha_t b_t(x_i)) = \sum_{i=1}^l exp(-y_i\sum_{t=1}^{T-1} \alpha_t b_t(x_i)) * exp(-y_i\alpha_T b_T(x_i)) $$

$exp(-y_i\sum_{t=1}^T-1 \alpha_t b_t(x_i))$ -- это вес $w_i$ i-го объекта.

Нормируем все веса и получаем вектор $\widetilde{W}^l = (\tilde{w_1},...,\tilde{w_l})$, где $\tilde{w}^i = \frac{w_i}{\sum_{j=1}^l w_j}$.

Определим два функционала качества:
- суммарный вес ошибочных классификаций $N(b; \widetilde{W}) = \sum_{i=1}^l\tilde{w}[b(x_i)=-y_i]$
- суммарный вес правильных классификаций $P(b; \widetilde{W}) = \sum_{i=1}^l\tilde{w}[b(x_i)=y_i]$

Пусть для любого нормированного вектора весов $\widetilde{W}$ существует базовый алгоритм $b$ классифицирующий выборку хотя бы немного лучше, чем на угад: $P(b; \widetilde{W})>N(b; \widetilde{W})$. Тогда минимум функционала $\widetilde{Q_T}$ достигается при: 
- $b_T = arg \underset{b}{max}\sqrt{P(b; \widetilde{W})} - \sqrt{N(b; \widetilde{W})}$
- $\alpha_T = \frac{1}{2} ln \frac{P(b; \widetilde{W})}{N(b; \widetilde{W})}$

**Алгоритм построения линейной комбинации классификаторов:**

*Вход:* 

- $X^l, Y^l$ - обучающая выборка;
- $T$ - максимальное число базовых алгоритмов;

*Выход:*
- базовые алгоритмы и их веса $\alpha_tb_t, \quad t=1,...,T$

*Алгоритм:*
1. инициализируем вектор весов: $w_i=1/l, \quad i=1,..l;$
2. для всех $t=1,...,T:$
- Обучить базовый алгоритм на наблюдениях $x_i \in X$, взятых с весом $w_i$;
- Определить суммарный вес правильных классификаций $P(b; \widetilde{W})$;
- Оценить важность алгоритма: $\alpha_t = \frac{1}{2} ln \frac{P(b; \widetilde{W})}{N(b; \widetilde{W})}$
- Пересчитать все объектов: $w_i = w_i* exp(-\alpha_t*y_i*b_t(x_i)), \quad i=1,...,l$
- Провести нормировку весов объектов.

## Градиентный бустинг

$$a_T(x)= \sum_{t=1}^T \alpha_t b_t(x), \quad x\in X, \quad b_t:X\rightarrow \mathbb{R}, \quad\alpha_t\in \mathbb{R}_+$$

**Эвристика:** обучаем $\alpha_t, b_t$ при фиксированных прилылущих.

**Критерий качества** с заданной гладкой функцией потерь $\mathcal{L}(b,y)$:
$$Q(\alpha;b;X^l)=\sum_{i=1}^l\mathcal{L}(\sum_{t=1}^{T-1}\alpha_t b_t(x_i)+\alpha b(x_i), y_i)\rightarrow\underset{\alpha, b}{min}$$
- $(a_{T-1,i})_{i=1}^l$ - вектор текущего приближения
- $(a_{T,i})_{i=1}^l$ - вектор следующего приближения

Градиентный метод минимизации $Q(F)\rightarrow min, f\in \mathbb{R}^l$:
- $a_{0,i}:=$ начальное приближение;
- $a_{T,i}:= a_{T-1,i}-\lambda g_i$ начальное приближение, $i=1,...,l$, где $g_i=\mathcal{L}_f'(a_{T-1,i}, y_i)$ - компоненты вектора градиента, $\lambda$- градиентный шаг;

А добавление одного базового алгоритма: 
$a_{T,i}:= a_{T-1,i}-\lambda b(x_i), \quad i=1,...,l$

Тогда будем искать базовый алгоритм, который приближает вектор антиградиента.

**Алгоритм градиентного бустинга:**

*Вход:* 

- $X^l, Y^l$ - обучающая выборка;
- $T$ - максимальное число базовых алгоритмов;
- $\lambda$ - скорость обучения.

*Выход:*
- базовые алгоритмы и их веса $\alpha_tb_t, \quad t=1,...,T$

*Алгоритм:*
1. инициализируем вектор начальный значений: $a_i=1/l, \quad i=1,..l;$
2. для всех $t=1,...,T:$
- Обучить базовый алгоритм, приближающий антиградиент:  
$b_t:=arg\underset{b\in B}{min}\sum_{i=1}^l(b(x_i)+\mathcal{L}'(w_i,y_i))^2$
- найти вес базового алгоритма $\alpha_t$:  
$\alpha_t:=arg\underset{a>0}{min}\sum_{i=1}^l\mathcal{L}(a_i+\lambda b_t(x_i);y_i)$
- обновить значения:  
$a_i:=a_i+\alpha_t b_t(x_i); \quad i=1,...,l$

**Алгоритм градиентного бустинга адаптация под логарифмическую функцию потерь:**
$$Q(\alpha;b;X^l) = \sum_{i=1}^l log_2(1+e^{-y_i*b(x_i) })\rightarrow \underset{w}{min}$$
Антиградиент: 

$\mathcal{L}'(b(x_i),y_i))= y_i*\sigma(-b(x_i)*y_i)$, где $\sigma(z) = \frac{1}{1+e^{-z}}$

*Вход:* 

- $X^l, Y^l$ - обучающая выборка;
- $T$ - максимальное число базовых алгоритмов;
- $\lambda$ - скорость обучения.

*Выход:*
- базовые алгоритмы и их веса $\alpha_tb_t, \quad t=1,...,T$
- в качестве базовой функции возьмем DecisionTreeRegressor()

*Алгоритм:*
1. инициализируем вектор начальный значений: $a_i=1/l, \quad i=1,..l;$
2. для всех $t=1,...,T:$
- Обучить базовый алгоритм на наблюдениях $\mathcal{L}'(b(x_i),y_i))$;  
- Обновить значения: 
$a_i:=a_i+\lambda b_t(x_i); \quad i=1,...,l$

In [1]:
from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler,RobustScaler
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor

In [2]:
X, y = make_classification(
    n_features=2, n_redundant=0, n_informative=2, random_state=1, n_clusters_per_class=1
)
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
linearly_separable = (X, y)

datasets = [
    make_moons(noise=0.3, random_state=0),
    make_circles(noise=0.2, factor=0.5, random_state=1),
    linearly_separable,
]

data_moon = datasets[1]

X, y = data_moon

y = np.array([ -1.0 if i == 0 else float(i) for i in y])

## Задание 

1. Реализовать алгоритм градиентного бустинга для логарифмической функции потерь.
2. Оценить качество работы алгоритма на сгенерированных трех наборах данных. 

In [3]:
def get_grid(data):
    x_min, x_max = data[:, 0].min() - 10, data[:, 0].max() + 10
    y_min, y_max = data[:, 1].min() - 10, data[:, 1].max() + 10
    return np.meshgrid(np.arange(x_min, x_max, 1), np.arange(y_min, y_max, 1))


t = [[(85, 174), (93, 156), (103, 176), (113, 152), (123, 133), (128, 160), (147, 127), (152, 137), (180, 124), (180, 148), (205, 133), (207, 113), (199, 158), (222, 168), (235, 137), (237, 152), (265, 149), (258, 180), (237, 196), (255, 214), (270, 203), (280, 186), (280, 228), (269, 239), (300, 203), (289, 240), (279, 270), (294, 271), (308, 238), (118, 187), (154, 154)], [(157, 226), (180, 205), (179, 238), (196, 225), (171, 256), (201, 255), (184, 288), (218, 254), (215, 293), (233, 282), (217, 330), (226, 316), (241, 314), (253, 337), (267, 314), (278, 335), (286, 349), (290, 314), (309, 345), (328, 347), (329, 314), (314, 322), (342, 286), (356, 314), (360, 332), (368, 286), (376, 307), (387, 273), (384, 248), (395, 289), (377, 263), (389, 213), (409, 224), (409, 258), (376, 202), (380, 169), (408, 186), (408, 205), (424, 164), (406, 149), (436, 197), (427, 242), (419, 273), (402, 313), (380, 335)]]
n1 = len(t[0])
n2 = len(t[1])

train_data = np.r_[t[0], t[1]]
train_labels = np.r_[np.ones(n1) * -1, np.ones(n2)]

XN = len(train_data)    # длина обучающей выборки
T = 10                  # число алгоритмов в композиции
max_depth = 4           # максимальная глубина решающих деревьев
w = np.ones(XN) / XN    # начальные значения весов для объектов выборки
algs = []               # список из полученных алгоритмов
alfa = []               # список из вычисленных весов для композиции

for n in range(T):
    algs.append(DecisionTreeClassifier(criterion='gini', max_depth=max_depth))
    algs[n].fit(train_data, train_labels, sample_weight=w)

    predicted = algs[n].predict(train_data) # формируем прогнозы полученного дерева по обучающей выборке
    N = np.sum(np.abs(train_labels - predicted) / 2 * w)   # вычисляем долю неверных классификаций
    alfa.append( 0.5 * np.log((1 - N) / N) if N != 0 else np.log((1-1e-8) / 1e-8) ) # вычисляем вес для текущего алгоритма

    # пересчитываем веса объектов выборки
    w = w * np.exp(-1 * alfa[n] * train_labels * predicted)
    w = w / np.sum(w)

**Алгоритм градиентного бустинга адаптация под логарифмическую функцию потерь:**
$$Q(\alpha;b;X^l) = \sum_{i=1}^l log_2(1+e^{-y_i*b(x_i) })\rightarrow \underset{w}{min}$$
Антиградиент: 

$\mathcal{L}'(b(x_i),y_i))= y_i*\sigma(-b(x_i)*y_i)$, где $\sigma(z) = \frac{1}{1+e^{-z}}$

*Вход:* 

- $X^l, Y^l$ - обучающая выборка;
- $T$ - максимальное число базовых алгоритмов;
- $\lambda$ - скорость обучения.

*Выход:*
- базовые алгоритмы и их веса $\alpha_tb_t, \quad t=1,...,T$
- в качестве базовой функции возьмем DecisionTreeRegressor()

*Алгоритм:*
1. инициализируем вектор начальный значений: $a_i=1/l, \quad i=1,..l;$
2. для всех $t=1,...,T:$
- Обучить базовый алгоритм на наблюдениях $\mathcal{L}'(b(x_i),y_i))$;  
- Обновить значения: 
$a_i:=a_i+\lambda b_t(x_i); \quad i=1,...,l$

**Алгоритм градиентного бустинга:**

*Вход:* 

- $X^l, Y^l$ - обучающая выборка;
- $T$ - максимальное число базовых алгоритмов;
- $\lambda$ - скорость обучения.

*Выход:*
- базовые алгоритмы и их веса $\alpha_tb_t, \quad t=1,...,T$

*Алгоритм:*
1. инициализируем вектор начальный значений: $a_i=1/l, \quad i=1,..l;$
2. для всех $t=1,...,T:$
- Обучить базовый алгоритм, приближающий антиградиент:  
$b_t:=arg\underset{b\in B}{min}\sum_{i=1}^l(b(x_i)+\mathcal{L}'(w_i,y_i))^2$
- найти вес базового алгоритма $\alpha_t$:  
$\alpha_t:=arg\underset{a>0}{min}\sum_{i=1}^l\mathcal{L}(a_i+\lambda b_t(x_i);y_i)$
- обновить значения:  
$a_i:=a_i+\alpha_t b_t(x_i); \quad i=1,...,l$

In [4]:
class LogLossGradientBoosting:
    def __init__(self, T=100, learning_rate=0.1):
        self.T = T  #максимальное число базовых алгоритмов
        self.learning_rate = learning_rate  # скорость обучения
        self.models = []  #для хранения базовых моделей
        self.alphas = []  #для хранения весов моделей

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def anti_gradient(self, b_x, y):
        return y * self.sigmoid(-b_x * y)

    def fit(self, X, y):
        # Инициализация начальных значений
        self.a = np.full(len(y), 1 / len(y))  

        for t in range(self.T):
            model = DecisionTreeRegressor()
            anti_grad = self.anti_gradient(self.a, y)
            model.fit(X, anti_grad)

            pred = model.predict(X)
            self.models.append(model)

            self.a += self.learning_rate * pred
            self.alphas.append(self.learning_rate * pred)

    def predict(self, X):
        total_prediction = np.zeros(X.shape[0])

        for model in self.models:
            total_prediction += model.predict(X)

        return total_prediction

In [5]:
model = LogLossGradientBoosting(T=100, learning_rate=0.1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=13)

In [7]:
model.fit(X_train, y_train)

In [8]:
y_pred = model.predict(X_test)

In [9]:
np.sign(y_pred)

array([ 1., -1., -1., -1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1.,  1.,
       -1.,  1.,  1.,  1., -1., -1., -1.])

In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,np.sign(y_pred))

0.6

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import RocCurveDisplay, roc_curve, roc_auc_score, auc
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline


In [12]:
df = pd.read_excel('orders_short_v2.xlsx')

df = df[(df['Контрибуция']>-60000000) & (df['Среднее время между редакциями']>=0)]

In [13]:
df_rule = df['Дата отправления']>='2023-09-01'

categorial = ['Оператор', 'Учетное направление', 'Кластер назначения', 'Кластер отправления',]
df.drop(['Unnamed: 0',
         'Идентификатор погрузки',
         'Дата редактирования', 
         'Дата отправления', 
         'Дата отправления факт', 
         'Количество редакций после отправки',
         'Длительность афтершока', 
         'Код назначения', 
         'Код отправления', 
         'Идентификатор переноса'], axis = 1, inplace = True)

numerical = ['версия погрузки', 
             'Количество вагонов', 
             'Контрибуция', 
             'Длительность жизни', 
             'Время до отправки',
             'Количество отмен', 
             'Количество переносов',
             'Среднее время между редакциями']

In [14]:
X_train = df[~df_rule].drop('Результат', axis = 1)
y_train = df['Результат'][~df_rule]

X_test = df[df_rule].drop('Результат', axis = 1)
y_test = df['Результат'][df_rule]

numerical_transformer = RobustScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical),
        ('cat', categorical_transformer, categorial)])

kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
model_gbc = GradientBoostingClassifier()

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', model_gbc)])

In [17]:
   parameters = {
       'classifier__learning_rate': np.arange(0.05,1,0.1),  # Пример значений
       'classifier__loss':['log_loss', 'exponential'],
       'classifier__max_depth':np.arange(2,8,1),
       'classifier__max_features':['sqrt','log2'],
       'classifier__min_samples_leaf':np.arange(1,6,1)
        }

In [18]:
clf = GridSearchCV(pipeline, parameters, cv = kf, verbose = 3, scoring = 'roc_auc')
clf.fit(X_train, y_train)

Fitting 5 folds for each of 1200 candidates, totalling 6000 fits
[CV 1/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=2, classifier__max_features=sqrt, classifier__min_samples_leaf=1;, score=0.832 total time=   0.1s
[CV 2/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=2, classifier__max_features=sqrt, classifier__min_samples_leaf=1;, score=0.797 total time=   0.1s
[CV 3/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=2, classifier__max_features=sqrt, classifier__min_samples_leaf=1;, score=0.824 total time=   0.1s
[CV 4/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=2, classifier__max_features=sqrt, classifier__min_samples_leaf=1;, score=0.798 total time=   0.1s
[CV 5/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=2, classifier__max_features=sqrt, classifier__min_samples_leaf=1;, score=

[CV 4/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=2, classifier__max_features=log2, classifier__min_samples_leaf=4;, score=0.789 total time=   0.1s
[CV 5/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=2, classifier__max_features=log2, classifier__min_samples_leaf=4;, score=0.813 total time=   0.1s
[CV 1/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=2, classifier__max_features=log2, classifier__min_samples_leaf=5;, score=0.817 total time=   0.1s
[CV 2/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=2, classifier__max_features=log2, classifier__min_samples_leaf=5;, score=0.801 total time=   0.1s
[CV 3/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=2, classifier__max_features=log2, classifier__min_samples_leaf=5;, score=0.818 total time=   0.1s
[CV 4/5] END classifier__learning_rate=0

[CV 2/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=3, classifier__max_features=log2, classifier__min_samples_leaf=3;, score=0.816 total time=   0.2s
[CV 3/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=3, classifier__max_features=log2, classifier__min_samples_leaf=3;, score=0.831 total time=   0.2s
[CV 4/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=3, classifier__max_features=log2, classifier__min_samples_leaf=3;, score=0.810 total time=   0.2s
[CV 5/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=3, classifier__max_features=log2, classifier__min_samples_leaf=3;, score=0.831 total time=   0.2s
[CV 1/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=3, classifier__max_features=log2, classifier__min_samples_leaf=4;, score=0.842 total time=   0.2s
[CV 2/5] END classifier__learning_rate=0

[CV 5/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=4, classifier__max_features=log2, classifier__min_samples_leaf=1;, score=0.843 total time=   0.3s
[CV 1/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=4, classifier__max_features=log2, classifier__min_samples_leaf=2;, score=0.850 total time=   0.2s
[CV 2/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=4, classifier__max_features=log2, classifier__min_samples_leaf=2;, score=0.825 total time=   0.2s
[CV 3/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=4, classifier__max_features=log2, classifier__min_samples_leaf=2;, score=0.848 total time=   0.2s
[CV 4/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=4, classifier__max_features=log2, classifier__min_samples_leaf=2;, score=0.811 total time=   0.2s
[CV 5/5] END classifier__learning_rate=0

[CV 3/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=5, classifier__max_features=sqrt, classifier__min_samples_leaf=5;, score=0.860 total time=   0.4s
[CV 4/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=5, classifier__max_features=sqrt, classifier__min_samples_leaf=5;, score=0.826 total time=   0.4s
[CV 5/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=5, classifier__max_features=sqrt, classifier__min_samples_leaf=5;, score=0.853 total time=   0.4s
[CV 1/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=5, classifier__max_features=log2, classifier__min_samples_leaf=1;, score=0.863 total time=   0.4s
[CV 2/5] END classifier__learning_rate=0.05, classifier__loss=log_loss, classifier__max_depth=5, classifier__max_features=log2, classifier__min_samples_leaf=1;, score=0.839 total time=   0.4s
[CV 3/5] END classifier__learning_rate=0

...

[CV 4/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=4, classifier__max_features=log2, classifier__min_samples_leaf=4;, score=0.825 total time=   0.3s
[CV 5/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=4, classifier__max_features=log2, classifier__min_samples_leaf=4;, score=0.824 total time=   0.3s
[CV 1/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=4, classifier__max_features=log2, classifier__min_samples_leaf=5;, score=0.860 total time=   0.3s
[CV 2/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=4, classifier__max_features=log2, classifier__min_samples_leaf=5;, score=0.838 total time=   0.1s
[CV 3/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=4, classifier__max_features=log2, classifier__min_sam

[CV 4/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=5, classifier__max_features=log2, classifier__min_samples_leaf=2;, score=0.803 total time=   0.3s
[CV 5/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=5, classifier__max_features=log2, classifier__min_samples_leaf=2;, score=0.809 total time=   0.3s
[CV 1/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=5, classifier__max_features=log2, classifier__min_samples_leaf=3;, score=0.856 total time=   0.2s
[CV 2/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=5, classifier__max_features=log2, classifier__min_samples_leaf=3;, score=0.834 total time=   0.3s
[CV 3/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=5, classifier__max_features=log2, classifier__min_sam

[CV 4/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=6, classifier__max_features=sqrt, classifier__min_samples_leaf=5;, score=0.826 total time=   0.3s
[CV 5/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=6, classifier__max_features=sqrt, classifier__min_samples_leaf=5;, score=0.841 total time=   0.3s
[CV 1/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=6, classifier__max_features=log2, classifier__min_samples_leaf=1;, score=0.860 total time=   0.4s
[CV 2/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=6, classifier__max_features=log2, classifier__min_samples_leaf=1;, score=0.832 total time=   0.4s
[CV 3/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=6, classifier__max_features=log2, classifier__min_sam

[CV 4/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=7, classifier__max_features=sqrt, classifier__min_samples_leaf=3;, score=0.814 total time=   0.5s
[CV 5/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=7, classifier__max_features=sqrt, classifier__min_samples_leaf=3;, score=0.826 total time=   0.4s
[CV 1/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=7, classifier__max_features=sqrt, classifier__min_samples_leaf=4;, score=0.871 total time=   0.4s
[CV 2/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=7, classifier__max_features=sqrt, classifier__min_samples_leaf=4;, score=0.835 total time=   0.4s
[CV 3/5] END classifier__learning_rate=0.9500000000000002, classifier__loss=exponential, classifier__max_depth=7, classifier__max_features=sqrt, classifier__min_sam

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         RobustScaler(),
                                                                         ['версия '
                                                                          'погрузки',
                                                                          'Количество '
                                                                          'вагонов',
                                                                          'Контрибуция',
                                                                          'Длительность '
                                                                          'жизни',
                                                                          'Время '
                                                                          'до '
                                                                          'отправки',
                                                                          'Количество '
                                                                          'отмен',
                                                                          'Количество '
                                                                          'переносов',
                                                                          'Среднее '
                                                                          'время '
                                                                          'между '
                                                                          'редакциями'])...
                                       ('classifier',
                                        GradientBoostingClassifier())]),
             param_grid={'classifier__learning_rate': array([0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95]),
                         'classifier__loss': ['log_loss', 'exponential'],
                         'classifier__max_depth': array([2, 3, 4, 5, 6, 7]),
                         'classifier__max_features': ['sqrt', 'log2'],
                         'classifier__min_samples_leaf': array([1, 2, 3, 4, 5])},
             scoring='roc_auc', verbose=3)

In [20]:
y_pred = clf.predict(X_test)

In [25]:
accuracy_score(y_test,y_pred)

0.7131782945736435

In [22]:
precision_score(y_test,y_pred)

0.7656903765690377

In [23]:
recall_score(y_test,y_pred)

0.7689075630252101

In [24]:
print(classification_report(y_test, y_pred, labels = clf.classes_))

              precision    recall  f1-score   support

           0       0.63      0.62      0.63       149
           1       0.77      0.77      0.77       238

    accuracy                           0.71       387
   macro avg       0.70      0.70      0.70       387
weighted avg       0.71      0.71      0.71       387



In [26]:
precision_score(y_train,clf.predict(X_train))

0.9266169154228856

In [27]:
accuracy_score(y_train,clf.predict(X_train))

0.9277498202731848

In [28]:
recall_score(y_train,clf.predict(X_train))

0.94723458359822

In [32]:
clf.best_params_

{'classifier__learning_rate': 0.15000000000000002,
 'classifier__loss': 'exponential',
 'classifier__max_depth': 7,
 'classifier__max_features': 'sqrt',
 'classifier__min_samples_leaf': 3}